Start by importing packages and data

In [49]:
import torch
import os
import csv
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from PIL import Image 
import torchvision.datasets as dset
import torchvision.transforms as T
import random
import numpy as np
import matplotlib.pyplot as plt
import statistics

In [50]:
# Data ingest

# Get all labels imported and indexed, with all whitespaces stripped out
labels_list = []
dataset = []
#import dataset to empty arrays of data and label
for (root, dirs, files) in os.walk("crop/"):
    labels_list = dirs
    for dire in dirs:
        curr_dir = root + dire
        for (rt,di,fi) in os.walk(curr_dir):
            for img in fi:
                data = {}
                data["image"] = T.functional.pil_to_tensor(Image.open(curr_dir+"/"+img))#.to(torch.float32)
                data["label"] = labels_list.index(dire)
                dataset.append(data)
                # for each file in the sub directory
                # append the image and corresponding label to dataset
    break

random.shuffle(labels_list)

In [3]:
print(len(dataset))

21143


In [4]:
heights = []
widths = []
for i in dataset:
    c,h,w = i["image"].shape
    heights.append(h)
    widths.append(w)

#plt.bar(4000,heights)
#plt.show()
print (statistics.mean(heights))
print (statistics.median(heights))
print (statistics.multimode(heights))
print (statistics.mean(widths))
print (statistics.median(widths))
print (statistics.multimode(widths))
print (max(heights))
print (max(widths))

340.69119803244575
203
[71]
702.7543867946838
453
[1199]
4911
8073


In [7]:
max_height = 4912
max_width  = 8080

In [133]:
class Patchwork (nn.Module):
    def __init__ (self, patch_size = 16):
        super().__init__()
        self.patch_size = patch_size
        self.num_patches = (max_height * max_width) // (self.patch_size ** 2)
        self.patch_dim = (self.patch_size ** 2) * 3
        self.linear_size = self.patch_dim // 2
        self.linear_layer = nn.Linear(self.patch_dim,self.linear_size)

    def forward(self,image):
        image = image.to(torch.float32)
        c,h,w = image.shape

        #size standardization
        patch_height = max_height - h
        patch_width = max_width - w

        patch_bottom = patch_height // 2
        if (patch_height % 2):
            patch_top = patch_bottom + 1
        else:
            patch_top = patch_bottom

        patch_right = patch_width // 2

        if (patch_width % 2):
            patch_left = patch_right + 1
        else:
            patch_left = patch_right

        padding = torch.nn.ZeroPad2d((patch_left,patch_right,patch_top,patch_bottom))
        image = padding(image)
        
        patches = []
        for i in range(self.num_patches):
            x_coor = i // (max_width // self.patch_size)
            y_coor = i - x_coor * (max_width // self.patch_size)

            patch = image[:, x_coor * self.patch_size: (x_coor + 1) * self.patch_size,y_coor * self.patch_size:(y_coor+1)*self.patch_size]
            patches.append(patch.flatten())
            
        patches = torch.stack(patches)
        patches = self.linear_layer(patches)
        classification_token = nn.Parameter(torch.rand(self.linear_size))
        patches = torch.vstack((classification_token,patches))

        return patches

In [134]:
test = Patchwork()
x = test(dataset[69]["image"])
print(x.shape)

torch.Size([155036, 384])
